<h1 align='center'>T20 WorldCup Cricket Data Pre Processing</h1>

In [1]:
import pandas as pd
import json

<h4 style ='color:blue'> (1) Process Match Results</h4>

In [2]:
with open('t20_wc_match_results.json') as f:
    data=json.load(f)

In [3]:
df_match = pd.DataFrame(data[0]['matchSummary'])
df_match.head()

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [4]:
df_match.shape

(45, 7)

**Use Scorecard as match id to link with other table**

In [5]:
df_match.rename({'scorecard':'match_id'}, axis=1, inplace=True)
df_match.head()

,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


**Create a match ids dictionary that maps team name with unique match id. This will later come handy when liking other tables**

In [6]:
match_ids_dict = {}
for iter, row in df_match.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + ' Vs ' + row['team1']
    match_ids_dict[key1] = row['match_id']
    match_ids_dict[key2] = row['match_id']
   


In [26]:
df_match.to_csv('dim_match_summary.csv', index=False)

<h4 style='color:blue'>(2) Process Batting Summary</h4>

In [7]:
with open('t20_wc_batting_summary.json') as f:
    data=json.load(f)
    all_record=[]
    for rec in data:
        all_record.extend(rec['battingSummary'])
all_record
df_batting = pd.DataFrame(all_record)    
df_batting.head(11)
       

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,run out (Gunathilaka/â€ Mendis),44,28,4,0,157.14
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,c â€ Mendis b Theekshana,0,1,0,0,0.00
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,,31,16,2,2,193.75
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,c Smit b Shikongo,9,10,1,0,90.00
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,c â€ Green b Wiese,6,6,0,0,100.00


In [8]:
df_batting['out/not_out'] = df_batting.dismissal.apply(lambda x: 'out' if len(x) > 0 else 'not_out')
df_batting.head(11)

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR,out/not_out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,0,0,50.00,out
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,1,0,100.00,out
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,c â€ Mendis b Karunaratne,20,12,1,2,166.66,out
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,2,0,108.33,out
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),c Gunathilaka b PWH de Silva,20,24,0,0,83.33,out
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,run out (Gunathilaka/â€ Mendis),44,28,4,0,157.14,out
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,c â€ Mendis b Theekshana,0,1,0,0,0.00,out
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,,31,16,2,2,193.75,not_out
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,c Smit b Shikongo,9,10,1,0,90.00,out
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,c â€ Green b Wiese,6,6,0,0,100.00,out


In [9]:
df_batting.drop(columns=['dismissal'],inplace=True)



In [10]:
df_batting.dtypes

match          object
teamInnings    object
battingPos      int64
batsmanName    object
runs           object
balls          object
4s             object
6s             object
SR             object
out/not_out    object
dtype: object

In [11]:
import re
pattern= r'[^aA-z)Z(\s]'

df_batting['BatsmanName'] = df_batting.batsmanName.apply(lambda x: re.sub(pattern,"",x))
df_batting.head(10)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,BatsmanName
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out,Michael van Lingen
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out,Divan la Cock
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,Jan Nicol LoftieEaton
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,Stephan Baard
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out,Gerhard Erasmus(c)
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,out,Jan Frylinck
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,out,David Wiese
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,not_out,JJ Smit
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,out,Pathum Nissanka
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,6,6,0,0,100.00,out,Kusal Mendis


In [12]:
df_batting.drop(columns='batsmanName', axis=1, inplace=True)

In [13]:
df_batting.rename({'BatsmanName':'batsmanName'}, axis=1,inplace=True)

In [14]:
df_batting.head(10)

,match,teamInnings,battingPos,runs,balls,4s,6s,SR,out/not_out,batsmanName
0,Namibia Vs Sri Lanka,Namibia,1,3,6,0,0,50.00,out,Michael van Lingen
1,Namibia Vs Sri Lanka,Namibia,2,9,9,1,0,100.00,out,Divan la Cock
2,Namibia Vs Sri Lanka,Namibia,3,20,12,1,2,166.66,out,Jan Nicol LoftieEaton
3,Namibia Vs Sri Lanka,Namibia,4,26,24,2,0,108.33,out,Stephan Baard
4,Namibia Vs Sri Lanka,Namibia,5,20,24,0,0,83.33,out,Gerhard Erasmus(c)
5,Namibia Vs Sri Lanka,Namibia,6,44,28,4,0,157.14,out,Jan Frylinck
6,Namibia Vs Sri Lanka,Namibia,7,0,1,0,0,0.00,out,David Wiese
7,Namibia Vs Sri Lanka,Namibia,8,31,16,2,2,193.75,not_out,JJ Smit
8,Namibia Vs Sri Lanka,Sri Lanka,1,9,10,1,0,90.00,out,Pathum Nissanka
9,Namibia Vs Sri Lanka,Sri Lanka,2,6,6,0,0,100.00,out,Kusal Mendis


In [15]:
df_batting['match_id'] = df_batting['match'].map(match_ids_dict)
df_batting.head()

,match,teamInnings,battingPos,runs,balls,4s,6s,SR,out/not_out,batsmanName,match_id
0,Namibia Vs Sri Lanka,Namibia,1,3,6,0,0,50.00,out,Michael van Lingen,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,9,9,1,0,100.00,out,Divan la Cock,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,20,12,1,2,166.66,out,Jan Nicol LoftieEaton,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,26,24,2,0,108.33,out,Stephan Baard,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,20,24,0,0,83.33,out,Gerhard Erasmus(c),T20I # 1823


In [16]:
df_batting.to_csv('fact_batting.csv', index=False)

<h4 style='color:blue'>(3)Process Bowling Summary</h4>

In [48]:
with open('t20_wc_bowling_summary.json') as f:
    data = json.load(f)
    all_records =[]
    for rec in data:
        all_records.extend(rec['bowlingSummary'])
        

all_records[:2]

[{'match': 'Namibia Vs Sri Lanka',
  'bowlingTeam': 'Sri Lanka',
  'bowlerName': 'Maheesh Theekshana',
  'overs': '4',
  'maiden': '0',
  'runs': '23',
  'wickets': '1',
  'economy': '5.75',
  '0s': '7',
  '4s': '0',
  '6s': '0',
  'wides': '2',
  'noBalls': '0'},
 {'match': 'Namibia Vs Sri Lanka',
  'bowlingTeam': 'Sri Lanka',
  'bowlerName': 'Dushmantha Chameera',
  'overs': '4',
  'maiden': '0',
  'runs': '39',
  'wickets': '1',
  'economy': '9.75',
  '0s': '6',
  '4s': '3',
  '6s': '1',
  'wides': '2',
  'noBalls': '0'}]

In [49]:
df_bowling = pd.DataFrame(all_records)
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0


In [50]:
df_bowling.shape

(500, 13)

In [51]:
df_bowling['match_id'] = df_bowling['match'].map(match_ids_dict)

In [53]:
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823


In [64]:
df_bowling['BowlerName'] = df_bowling.bowlerName.apply(lambda x: re.sub(pattern,"",x))

In [67]:
df_bowling.drop(columns={'bowlerName'},axis=1,inplace=True)

In [69]:
df_bowling.rename({'BowlerName':'bowlerName'},axis=1, inplace=True)

In [70]:
df_bowling.head(20)

,match,bowlingTeam,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id,bowlerName
0,Namibia Vs Sri Lanka,Sri Lanka,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823,Maheesh Theekshana
1,Namibia Vs Sri Lanka,Sri Lanka,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823,Dushmantha Chameera
2,Namibia Vs Sri Lanka,Sri Lanka,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823,Pramod Madushan
3,Namibia Vs Sri Lanka,Sri Lanka,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823,Chamika Karunaratne
4,Namibia Vs Sri Lanka,Sri Lanka,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823,Wanindu Hasaranga de Silva
5,Namibia Vs Sri Lanka,Namibia,1,0,8,0,8.00,1,1,0,0,0,T20I # 1823,Gerhard Erasmus
6,Namibia Vs Sri Lanka,Namibia,4,0,16,2,4.00,13,1,0,0,0,T20I # 1823,David Wiese
7,Namibia Vs Sri Lanka,Namibia,4,0,18,2,4.50,10,1,0,0,0,T20I # 1823,Bernard Scholtz
8,Namibia Vs Sri Lanka,Namibia,3,1,22,2,7.33,6,3,0,0,0,T20I # 1823,Ben Shikongo
9,Namibia Vs Sri Lanka,Namibia,3,0,16,1,5.33,7,0,0,1,0,T20I # 1823,JJ Smit


In [71]:
df_bowling.to_csv('fact_bowling_summary.csv')

<h4 style='color:blue'>(4)Process Players Information<h4>

In [60]:
with open('t20_wc_player_info.json') as f:
    data=json.load(f)
df_players = pd.DataFrame(data)

In [63]:
df_players.head(10)

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder,
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder,David Wiese joined a marked outflow of South A...
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder,
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter,
9,Kusal Mendisâ€,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter,"Blessed with a compact technique, an aggressiv..."


In [75]:
df_players['Name']= df_players.name.apply(lambda x: re.sub(pattern, "", x))

In [76]:
df_players.head()

,name,team,battingStyle,bowlingStyle,playingRole,description,Name
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,,Michael van Lingen
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,,Divan la Cock
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,,Jan Nicol LoftieEaton
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,,Stephan Baard
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,,Gerhard Erasmus(c)


In [77]:
df_players.drop(columns={'name'}, axis=1, inplace=True)

In [79]:
df_players.rename({'Name':'name'}, axis=1, inplace=True)

In [80]:
df_players.head()

,team,battingStyle,bowlingStyle,playingRole,description,name
0,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,,Michael van Lingen
1,Namibia,Right hand Bat,Legbreak,Opening Batter,,Divan la Cock
2,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,,Jan Nicol LoftieEaton
3,Namibia,Right hand Bat,Right arm Medium fast,Batter,,Stephan Baard
4,Namibia,Right hand Bat,Right arm Offbreak,Allrounder,,Gerhard Erasmus(c)


In [81]:
df_players[df_players['team']=='India']

,team,battingStyle,bowlingStyle,playingRole,description,name
127,India,Right hand Bat,,Opening Batter,"A tall, elegant right-hand batsman who can kee...",KL Rahul
128,India,Right hand Bat,Right arm Offbreak,Top order Batter,"Languid and easy on the eye, Rohit Sharma owne...",Rohit Sharma(c)
129,India,Right hand Bat,Right arm Medium,Top order Batter,India has given to the world many a great cric...,Virat Kohli
130,India,Right hand Bat,"Right arm Medium, Right arm Offbreak",Batter,Hard-hitting 360-degree batter Suryakumar Yada...,Suryakumar Yadav
131,India,Left hand Bat,Slow Left arm Orthodox,Bowling Allrounder,Left-arm spinner Axar Patel has been increasin...,Axar Patel
132,India,Right hand Bat,Right arm Medium fast,Allrounder,Hardik Pandya swears by living life king size ...,Hardik Pandya
133,India,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,Not many would forget the sight of Dinesh Kart...,Dinesh Karthik
134,India,Right hand Bat,Right arm Offbreak,Bowling Allrounder,R Ashwin took the tricks and skills he learned...,Ravichandran Ashwin
135,India,Right hand Bat,Right arm Medium,Bowler,"At the time of his India debut in 2012, Bhuvne...",Bhuvneshwar Kumar
136,India,Left hand Bat,Left arm Medium fast,Bowler,,Arshdeep Singh


In [82]:
df_players.to_csv('dim_players_no_images.csv', index=False)